In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
#thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage
from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.activity as activity
from lib.common.visualization import merge_image_sequence_movie

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [3]:
dataDB = DataFCDatabase(params)

Reading channel label file
Reading channel area file
Reading allen brain map
Reading task structure
Reading session structure
Searching for data files
Found mice ['mou_5', 'mou_6', 'mou_7', 'mou_9']


In [4]:
ds = DataStorage('gallerosalas_result_activity.h5')

In [5]:
mc = MetricCalculator(serial=True, verbose=False)

In [6]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

mice {'mou_9', 'mou_7', 'mou_6', 'mou_5'}
nSessions 4
datatypes ['raw', 'bn_session', 'bn_trial']
nChannel 27


In [7]:
dataDB.calc_shortest_distances()

In [9]:
dataDB.get_metadata('2017_03_06_session01')

,time_stamp,stimulus_time,reward_time,decision_time,end_time,trialType,Activity
0,14:58:05.0313,3145.1,8988.5,8989.5,11676.6,Hit,prior_move
1,14:58:20.8752,2885.1,Early,Early,10246.6,Early,None
2,14:58:35.2460,2874.2,NaN,9595.5,12350.7,CR,prior_move
3,14:58:51.7219,2890.2,7987.5,7988.5,10615.6,Hit,prior_move
4,14:59:06.4588,2872.1,7863.4,7864.4,10493.6,Hit,noisy
...,...,...,...,...,...,...,...
249,16:02:24.0580,2873.2,8073.4,8075.4,10713.6,Hit,quiet_then_move
250,16:02:38.8978,2874.2,Early,Early,11566.7,Early,None
251,16:02:54.5877,2864.2,7866.5,7868.5,10536.6,Hit,quiet_then_move
252,16:03:09.2526,2873.1,NaN,9591.5,12344.7,CR,prior_move


## Compute mean over time for all mice and condition combinations

In [11]:
argSweepDict = {
    'datatype' : ['bn_trial', 'bn_session'],
    'intervName' : 'auto'
}

In [12]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
]

In [ ]:
activity.compute_mean_interval(dataDB, ds, {'Hit', 'CR'}, skipExisting=False, exclQueryLst=exclQueryLst,
                               **argSweepDict)

# 1. Significance

## 1.1 Plot Hit activity by channel

In [ ]:
argSweepDict = {
    'datatype' : ['bn_trial', 'bn_session'],
    'intervName' : 'auto',
    'trialType' : 'auto'
}

In [ ]:
activity.activity_brainplot_mouse(dataDB, 'intervName', exclQueryLst=exclQueryLst, vmin=-0.02, vmax=0.02,
                                  **argSweepDict)

In [ ]:
activity.activity_brainplot_mouse(dataDB, 'trialType', exclQueryLst=exclQueryLst, vmin=-0.02, vmax=0.02,
                                  **argSweepDict)

In [ ]:
activity.activity_brainplot_mousephase_subpre(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02,
                                              intervName='auto', trialType='auto')

In [ ]:
activity.activity_brainplot_mousephase_submouse(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02, **argSweepDict)

## 1.2. Accuracy of hit vs cr prediction by mouse, phase

In [ ]:
activity.significance_brainplot_mousephase_byaction(dataDB, ds, performance=None,
                                                    metric='accuracy', minTrials=10,
                                                    limits=(0.5, 1.0))

In [ ]:
# activity.plot_classification_accuracy_bychannel(dataDB, drop6=True)
activity.classification_accuracy_brainplot_mousephase(dataDB, exclQueryLst, **argSweepDict)

## 1.3. Plot activity movies by chanel

In [14]:
activity.activity_brainplot_movie_mousetrialtype(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02,
                                                 haveDelay=True, trialType='auto',
                                                 datatype=['bn_trial', 'bn_session'])

Reading data,  bn_session mou_5 Hit
Reading data,  bn_session mou_5 Miss
Reading data,  bn_session mou_5 CR
Reading data,  bn_session mou_5 FA
Reading data,  bn_session mou_6 Hit
Reading data,  bn_session mou_6 Miss
Reading data,  bn_session mou_6 CR


Reading data,  bn_session mou_6 FA
Reading data,  bn_session mou_7 Hit
Reading data,  bn_session mou_7 Miss
Reading data,  bn_session mou_7 CR
Reading data,  bn_session mou_7 FA
Reading data,  bn_session mou_9 Hit


Reading data,  bn_session mou_9 Miss
Reading data,  bn_session mou_9 CR
Reading data,  bn_session mou_9 FA


IntProgress(value=0, description='bn_session', max=160)

Already calculated 0 skipping
Already calculated 1 skipping
Already calculated 2 skipping
Already calculated 3 skipping


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [15]:
merge_image_sequence_movie('activity_brainplot_mousetrialtype_bn_trial_', '.png', 0, 160,
                           trgPathName=None, deleteSrc=True)

Writing video to activity_brainplot_mousetrialtype_bn_trial_
Processing image[159]
 Done


In [16]:
merge_image_sequence_movie('activity_brainplot_mousetrialtype_bn_session_', '.png', 0, 160,
                           trgPathName=None, deleteSrc=True)

Writing video to activity_brainplot_mousetrialtype_bn_session_
Processing image[159]
 Done


# 2. Consistency

## 2.1. Consistency of channel activity

* DF/F should be independent of channel brightness/opacity, since it normalizes baseline.
* Thus, should be able to compare DFF of different mice directly

## 2.2 Channels significant for Hit/Cr. Overlap by mouse

1. Loop over datatype, interval, session
2. Test each channel for Hit/Cr
3. Count significant sessions for each channel
4. Scatter counts

**TODO**
* Split by Naive/Expert

In [ ]:
activity.plot_consistency_significant_activity_byaction(dataDB, ds)

## 2.3. Channels significant for phase. Overlap by mouse

In [ ]:
activity.plot_consistency_significant_activity_byphase(dataDB, ds, ['TEX', 'DEL'])